## Auto Shop GmbH Installation Script
Dieses Notebook enthält die notwendigen Befehle zur Installation der Auto Shop GmbH Applikation mittels Helm auf Kubernetes.


### Git Repository klonen

In [ ]:
%%bash
git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/helm.git
cd helm

### Kubernetes Namespaces erstellen und Helm Installation durchführen

In [ ]:
%%bash
cd helm
export KVMHOST="$(cat ~/work/server-ip)"

helm install autoshop-ch ./autoshop --namespace m01-ch --create-namespace --set ingress.host=ch.${KVMHOST}-edutbz.com --set image.tag=2.1.0

helm install autoshop-at ./autoshop --namespace m01-at --create-namespace --set ingress.host=at.${KVMHOST}-edutbz.com --set image.tag=2.0.0

helm install autoshop-d ./autoshop  --namespace m01-d  --create-namespace --set ingress.host=d.${KVMHOST}-edutbz.com  --set image.tag=1.0.0

**Hinweis**: Kubernetes-Zertifikate werden verwendet. Testen durch Aufruf der von Helm ausgegebenen URLs.

In [ ]:
%%bash
helm list -A

Kontrolle ob alle Ressourcen sauber erstellt wurden:

In [ ]:
%%bash
kubectl get all,ingress -n m01-ch

Mittels `curl` und `-H "Host:"` können wir gezielt einen URL ansprechen

In [ ]:
%%bash
export KVMHOST="$(cat ~/work/server-ip)"
curl -k -H "Host: ch.${KVMHOST}-edutbz.com" https://localhost:443/webshop

## DNS Server Einträge

Damit die Applikationen jeweils unter
* ch.[fqdn]
* at.[fqdn]
* d.[fqdn]

ansprechbar sind müssen folgende Einträge erfasst werden:

In [ ]:
%%bash
echo "Domain : $(cat ~/work/server-ip)-edutbz.com"
echo "- Record: ch, $(hostname -I | cut '-d ' -f1)"
echo "- Record: at, $(hostname -I | cut '-d ' -f1)"
echo "- Record: d, $(hostname -I | cut '-d ' -f1)"

Für `/etc/hosts` sind die Einträge wie folgt:

In [ ]:
%%bash
echo "$(cat ~/work/server-ip) ch.$(cat ~/work/server-ip)-edutbz.com"
echo "$(cat ~/work/server-ip) at.$(cat ~/work/server-ip)-edutbz.com"
echo "$(cat ~/work/server-ip)d.$(cat ~/work/server-ip)-edutbz.com"

### Fehleranalyse läuft ein Ingress Dienst?

Dann sollte mindestens eine `ingressClass` vorhanden sein.

In [ ]:
%%bash
kubectl get ingressClasses